<a href="https://colab.research.google.com/github/nyanta012/demo/blob/main/pdf_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install langchain==0.0.205 openai==0.27.2 pypdf==3.7.0 pdfminer.six==20221105 pymupdf==1.22.5

In [ ]:
from langchain.document_loaders import PDFMinerLoader
from langchain.document_loaders import PyMuPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import PyPDFLoader

from langchain.llms import OpenAIChat
import os
import json
import openai
import getpass
apikey = getpass.getpass(prompt = 'OpenAIのAPIキーを入力してください')

os.environ["OPENAI_API_KEY"] = apikey

OpenAIのAPIキーを入力してください··········


In [ ]:
path_pdf = "医療施設動態調査.pdf"

In [ ]:
llm = OpenAIChat(temperature=0.0, model_name="gpt-3.5-turbo")

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:748: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [ ]:
def pdf_parser(pdf_input):
    TEMPLATE = """
    以下に示すデータは、PDFをテキスト化したものである。
    種類別の施設数及び病床数の令和5年4月と3月のデータをJSONデータに変換せよ。

    # データ
    {}
    """.format(
        pdf_input
    )
    return llm(TEMPLATE)

# PyPDF

Llama indexのPDFReaderは、こちらを使っている

https://github.com/jerryjliu/llama_index/blob/3ea488a5c814fbe78bf593209a0199529b6e17e0/llama_index/readers/file/docs_reader.py#L28

In [ ]:
loader = PyPDFLoader(path_pdf)
data = loader.load_and_split()

In [ ]:
data[0]

Document(page_content='医療施設動態調査（令和５年４月末概数） \n \n病院の施設数は前月に比べ   ４施設の減少、病床数は  ９３２床の減少。 \n一般診療所の施設数は    ９４施設の減少、病床数は  ６８４床の減少。 \n歯科診療所の施設数は   １２１施設の減少、病床数は     増減なし。  \n \n１．種類別にみた施設数及び病床数 \n\u3000各月末現在\n施設数 病床数\n令和５年４月令和５年３月 令和５年４月令和５年３月\n総数          180 564         180 783      △    219総数        1 566 740       1 568 356      △  1 616\n 病院            8 135           8 139      △      4 病院        1 488 611       1 489 543      △    932\n\u3000 精神科病院  1 056  1 055              1\u3000 精神病床  320 623  321 019      △    396\n\u3000 一般病院  7 079  7 084      △      5\u3000 感染症病床  1 910  1 910  -\n\u3000 療養病床を有する病院（再掲）  3 434  3 439      △      5\u3000 結核病床  3 844  3 849      △      5\n\u3000 地域医療支援病院（再掲）   674             674  -\u3000 療養病床  276 373  276 604      △    231\n\u3000 一般病床  885 861  886 161      △    300\n 一般診療所          105 119         105 213      △     94 一般診療所  78 071          78 755      △    684\n\u3000 有床  5 790           5 834      △     44\n\u3000\u3000\u3000療養病床を有する\n\u3000

In [ ]:
print(data[0].page_content)

医療施設動態調査（令和５年４月末概数） 
 
病院の施設数は前月に比べ   ４施設の減少、病床数は  ９３２床の減少。 
一般診療所の施設数は    ９４施設の減少、病床数は  ６８４床の減少。 
歯科診療所の施設数は   １２１施設の減少、病床数は     増減なし。  
 
１．種類別にみた施設数及び病床数 
　各月末現在
施設数 病床数
令和５年４月令和５年３月 令和５年４月令和５年３月
総数          180 564         180 783      △    219総数        1 566 740       1 568 356      △  1 616
 病院            8 135           8 139      △      4 病院        1 488 611       1 489 543      △    932
　 精神科病院  1 056  1 055              1　 精神病床  320 623  321 019      △    396
　 一般病院  7 079  7 084      △      5　 感染症病床  1 910  1 910  -
　 療養病床を有する病院（再掲）  3 434  3 439      △      5　 結核病床  3 844  3 849      △      5
　 地域医療支援病院（再掲）   674             674  -　 療養病床  276 373  276 604      △    231
　 一般病床  885 861  886 161      △    300
 一般診療所          105 119         105 213      △     94 一般診療所  78 071          78 755      △    684
　 有床  5 790           5 834      △     44
　　　療養病床を有する
　　　一般診療所（再掲）  541             554      △     13　 療養病床（再掲）  5 289  5 437      △    148
　 無床  99 329          99 379     

In [ ]:
response = pdf_parser(data[0].page_content)

In [ ]:
print(response)

{
  "令和5年4月": {
    "種類別施設数": {
      "総数": {
        "施設数": "180",
        "病床数": "564"
      },
      "病院": {
        "施設数": "8",
        "病床数": "135"
      },
      "精神科病院": {
        "施設数": "1",
        "病床数": "056"
      },
      "一般病院": {
        "施設数": "7",
        "病床数": "079"
      },
      "療養病床を有する病院（再掲）": {
        "施設数": "3",
        "病床数": "434"
      },
      "地域医療支援病院（再掲）": {
        "施設数": "674",
        "病床数": "0"
      },
      "一般病床": {
        "施設数": "885",
        "病床数": "861"
      },
      "一般診療所": {
        "施設数": "105",
        "病床数": "119"
      },
      "有床": {
        "施設数": "5",
        "病床数": "790"
      },
      "療養病床を有する一般診療所（再掲）": {
        "施設数": "541",
        "病床数": "0"
      },
      "無床": {
        "施設数": "99",
        "病床数": "329"
      },
      "歯科診療所": {
        "施設数": "67",
        "病床数": "310"
      }
    },
    "開設者別施設数": {
      "病院": {
        "施設数": "8",
        "病床数": "135"
      },
      "一般診療所": {
        "施設数": "105",
        "病床数": "

# PDFMiner

テキストの正確な位置や他のレイアウト情報（フォントなど）を取得できるらしい  
langchainのPDFMinerLoaderでは、ページ数を取得することができなさそう

In [ ]:
loader = PDFMinerLoader(path_pdf)
data = loader.load()

In [ ]:
print(data[0].page_content)

総数

 病院

　 精神科病院

　 一般病院

　 療養病床を有する病院（再掲）

 1 056

 7 079

 3 434

令和５年６月30 日 
【照会先】 

政策統括官付参事官付保健統計室 

室長      高山  研 
室長補佐  中内  健治 
医療施設統計第一係 

(代表電話）03（5253）1111 (内線 7520・7521) 
(直通電話）03（3595）2958 

          医 療 施 設 動 態 調 査 （ 令 和 ５ 年 ４ 月 末 概 数 ）  

病院の施設数は前月に比べ      ４施設の減少、病床数は    ９３２床の減少。 
一般診療所の施設数は        ９４施設の減少、病床数は    ６８４床の減少。 
歯科診療所の施設数は      １２１施設の減少、病床数は          増減なし。 

１．種類別にみた施設数及び病床数 

施設数

令和５年４月

令和５年３月

増減数

病床数

令和５年４月

令和５年３月

　各月末現在

増減数

         180 564          180 783

      △    219 総数

       1 566 740        1 568 356

      △  1 616

           8 135            8 139

      △      4  病院

       1 488 611        1 489 543

      △    932

 1 055

              1 　 精神病床

 320 623

 321 019

      △    396

　 地域医療支援病院（再掲）

  674              674

 - 　 療養病床

　 一般病床

 7 084

      △      5 　 感染症病床

 3 439

      △      5 　 結核病床

 1 910

 3 844

 276 373

 885 861

 1 910

 -

 3 849

      △      5

 276 604

      △    231

 886 161

      △    300

 一般診療所



In [ ]:
response = pdf_parser(data[0].page_content[:3000])

In [ ]:
print(response)

{
  "令和5年4月": {
    "病院": 1056,
    "精神科病院": 7079,
    "一般病院": 3434,
    "療養病床を有する病院（再掲）": 674,
    "病床数": {
      "総数": 1566740,
      "精神病床": 320623,
      "地域医療支援病院（再掲）": 674,
      "一般病床": 7084,
      "感染症病床": 3439,
      "結核病床": 1910
    }
  },
  "令和5年3月": {
    "病院": 1055,
    "精神科病院": 7084,
    "一般病院": 3444,
    "療養病床を有する病院（再掲）": 674,
    "病床数": {
      "総数": 1568356,
      "精神病床": 321019,
      "地域医療支援病院（再掲）": 674,
      "一般病床": 7089,
      "感染症病床": 3444,
      "結核病床": 1910
    }
  }
}


# PyMuPDF

C言語とPythonで実装されていてテキスト抽出速度が速く、様々なドキュメント形式がサポートされていて強そう  
https://pymupdf.readthedocs.io/en/latest/about.html#performance

In [ ]:
loader = PyMuPDFLoader(path_pdf)
data = loader.load()

In [ ]:
print(data[0].page_content)

 
        
 
 
 
 
         
 
     医療施設動態調査（令和５年４月末概数） 
 
病院の施設数は前月に比べ   ４施設の減少、病床数は  ９３２床の減少。 
一般診療所の施設数は    ９４施設の減少、病床数は  ６８４床の減少。 
歯科診療所の施設数は   １２１施設の減少、病床数は     増減なし。 
 
 
１．種類別にみた施設数及び病床数 
　各月末現在
施設数
病床数
令和５年４月
令和５年３月
令和５年４月
令和５年３月
総数
         180 564          180 783
      △    219 総数
       1 566 740        1 568 356
      △  1 616
 病院
           8 135            8 139
      △      4  病院
       1 488 611        1 489 543
      △    932
　 精神科病院
 1 056
 1 055
              1 　 精神病床
 320 623
 321 019
      △    396
　 一般病院
 7 079
 7 084
      △      5 　 感染症病床
 1 910
 1 910
 -
　 療養病床を有する病院（再掲）
 3 434
 3 439
      △      5 　 結核病床
 3 844
 3 849
      △      5
　 地域医療支援病院（再掲）
  674              674
 - 　 療養病床
 276 373
 276 604
      △    231
　 一般病床
 885 861
 886 161
      △    300
 一般診療所
         105 119          105 213
      △     94  一般診療所
 78 071           78 755
      △    684
　 有床
 5 790            5 834
      △     44
　　　療養病床を有する
　　　一般診療所（再掲）
  541              554
      △     13 

In [ ]:
response = pdf_parser(data[0].page_content)

In [ ]:
print(response)

{
  "令和5年4月": {
    "病院": {
      "施設数": 8135,
      "病床数": 1488611
    },
    "一般診療所": {
      "施設数": 105119,
      "病床数": 78071
    },
    "歯科診療所": {
      "施設数": 67310,
      "病床数": 58
    }
  },
  "令和5年3月": {
    "病院": {
      "施設数": 8139,
      "病床数": 1489543
    },
    "一般診療所": {
      "施設数": 105213,
      "病床数": 78755
    },
    "歯科診療所": {
      "施設数": 67431,
      "病床数": 58
    }
  }
}


In [ ]:
def pdf_parser(pdf_input):
    TEMPLATE = """
    以下に示すデータは、PDFをテキスト化したものである。
    種類別の施設数の令和5年4月と3月のデータをJSONデータに変換せよ。
    病院の施設数は、精神科病院、一般病院、療養病床を有する病院、地域医療支援病院がある
    一般診療所は有床、無床があるため、それぞれ取得せよ。

    # データ
    {}
    """.format(
        pdf_input
    )
    return llm(TEMPLATE)

In [ ]:
response = pdf_parser(data[0].page_content)

In [ ]:
print(response)

{
  "令和5年4月": {
    "病院": {
      "総数": 8135,
      "精神科病院": 1056,
      "一般病院": 7079,
      "療養病床を有する病院": 3434,
      "地域医療支援病院": 674
    },
    "一般診療所": {
      "総数": 105119,
      "有床": 5790,
      "療養病床を有する一般診療所": 541,
      "無床": 99329
    },
    "歯科診療所": 67310
  },
  "令和5年3月": {
    "病院": {
      "総数": 8139,
      "精神科病院": 1055,
      "一般病院": 7084,
      "療養病床を有する病院": 3439,
      "地域医療支援病院": 674
    },
    "一般診療所": {
      "総数": 105213,
      "有床": 5834,
      "療養病床を有する一般診療所": 554,
      "無床": 99379
    },
    "歯科診療所": 67431
  }
}


In [ ]:
from pandas import json_normalize
import json
import pandas as pd

response =json.loads(response)

list_data = []
for col in response.keys():
    list_data.append(json_normalize(response[col]))

In [ ]:
df = pd.concat(list_data).T
df.columns = response.keys()

In [ ]:
df

,令和5年4月,令和5年3月
歯科診療所,67310,67431
病院.総数,8135,8139
病院.精神科病院,1056,1055
病院.一般病院,7079,7084
病院.療養病床を有する病院,3434,3439
病院.地域医療支援病院,674,674
一般診療所.総数,105119,105213
一般診療所.有床,5790,5834
一般診療所.療養病床を有する一般診療所,541,554
一般診療所.無床,99329,99379


In [ ]:
df.to_csv("data.csv", encoding="shift_jis")